# Handling Missing Data

## Trade-Offs in Missing Data Conventions

**Two strategies:** using a mask that globally indicates missing values, or choosing a sentinel value that indicates a missing entry.

Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point NaN value, and the Python None object.

## Missing Data in Pandas

In [10]:
import numpy as np
import pandas as pd

In [11]:
# dtype=object means the best common type representation NumPy
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [12]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
53.9 ms ± 4.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.45 ms ± 58.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



In [14]:
vals1.sum() # cannot use sum because of none type

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### NaN: Missing numerical data

In [15]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

In [16]:
1 + np.nan

nan

In [17]:
0 *  np.nan

nan

In [18]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

In [19]:
# NumPy does provide some special aggregations that will ignore these missing values
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

### NaN and None in Pandas

In [20]:
pd.Series([1, np.nan, 2, None]) # always set as None as Nan for numbers

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [21]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [22]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

In [23]:
pd.Series(['hello', 'to', 'you', None])

0    hello
1       to
2      you
3     None
dtype: object

| Typeclass | Conversion When Storing NAs | NA Sentinel Value |
|:---------:|:---------------------------:|:-----------------:|
| floating 	| No change | np.nan |
| object 	| No change |	None or np.nan |
| integer 	| Cast to float64 |	np.nan |
| boolean 	| Cast to object | 	None or np.nan |

## Operating on Null Values

* isnull(): Generate a boolean mask indicating missing values
* notnull(): Opposite of isnull()
* dropna(): Return a filtered version of the data
* fillna(): Return a copy of the data with missing values filled or imputed


### Detecting null values

In [24]:
data = pd.Series([1, np.nan, 'hello', None])

In [25]:
data.isnull()


0    False
1     True
2    False
3     True
dtype: bool

In [26]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Dropping null values

In [27]:
data.dropna()


0        1
2    hello
dtype: object

In [28]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [29]:
# cannot drop single values from a DataFrame; we can only drop full rows or full columns
df.dropna()

,0,1,2
1,2.0,3.0,5


In [30]:
df.dropna(axis='columns') # axis = 1


,2
0,2
1,5
2,6


In [31]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [32]:
# only drop full row of NA
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [33]:
# thresh : specify a minimum number of non-null values for the row/column
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


#### Filling null values

In [34]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [35]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [36]:
# propagate the previous value forward
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [37]:
# propagate the next values backward
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [38]:
# DataFrame example
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [39]:
df.fillna(method='ffill', axis='columns') # axis = 1

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
